In [ ]:

from zipfile import ZipFile
file_name='/content/MIt-BIH Datasets-20220507T173735Z-001.zip'
with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [ ]:
import pandas as pd
MIT_BIH_train = pd.read_csv("/content/MIt-BIH Datasets/MIT-BIH-train.csv" , header = None)
MIT_BIH_validate = pd.read_csv("/content/MIt-BIH Datasets/MIT-BIH-validate.csv" , header = None)
test = pd.read_csv("/content/MIt-BIH Datasets/MIT-BIH-test.csv" , header = None)

In [ ]:
combine = [MIT_BIH_train , MIT_BIH_validate]

In [ ]:
train = pd.concat(combine , axis = 0 )

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings

import keras
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D
from tensorflow.keras.optimizers import Adam 
from keras.callbacks import LearningRateScheduler

import itertools
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, label_ranking_average_precision_score, label_ranking_loss, coverage_error

from tensorflow.keras.layers import BatchNormalization

import imblearn
from collections import Counter

from sklearn.utils import resample

warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.optimizers import Adam , Adadelta

In [ ]:
train.rename(columns={187:"Class"}, inplace=True)
test.rename(columns={187:"Class"}, inplace=True)

In [ ]:
mapping = {     0. : 'Normal Beat',
               1. : 'Supraventricular premature beat',
               2. : 'Premature ventricular contraction',
               3. : 'Fusion of ventricular',
               4. : 'Unclassifiable beat'
          }

In [ ]:
counter = Counter(train['Class'])
print(counter)

Counter({0.0: 89788, 15.0: 7857, 2.0: 7721, 1.0: 5288, 8.0: 3819, 4.0: 2532, 16.0: 1297, 10.0: 653, 12.0: 177, 6.0: 129, 14.0: 93, 5.0: 77, 17.0: 31, 11.0: 21, 7.0: 3})


In [ ]:
df_15=train[train['Class']==15]
df_2=train[train['Class']==2]
df_1=train[train['Class']==1]
df_8=train[train['Class']==8]
df_4=train[train['Class']==4]
df_16=train[train['Class']==16]
df_10=train[train['Class']==10]
df_12=train[train['Class']==12]
df_6=train[train['Class']==6]
df_14=train[train['Class']==14]
df_5=train[train['Class']==5]
df_17=train[train['Class']==17]
df_11=train[train['Class']==11]
df_7=train[train['Class']==7]


df_0=(train[train['Class']==0]).sample(n=20000,random_state=42)


train=pd.concat([df_0,df_15,df_2,df_1,df_8,df_4,df_16,df_10,df_12,df_6,df_14,df_5,df_17,df_11,df_7])

In [ ]:
tain_count_classes_resamble =train["Class"].value_counts()

In [ ]:
X_train=train.iloc[:,:186].values
X_test=test.iloc[:,:186].values

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE(k_neighbors=2)
X_train, y_train = oversample.fit_resample(X_train, train["Class"])


In [ ]:
counter = Counter(y_train)
print(counter)

Counter({0.0: 20000, 15.0: 20000, 2.0: 20000, 1.0: 20000, 8.0: 20000, 4.0: 20000, 16.0: 20000, 10.0: 20000, 12.0: 20000, 6.0: 20000, 14.0: 20000, 5.0: 20000, 17.0: 20000, 11.0: 20000, 7.0: 20000})


In [ ]:
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_test = X_test.reshape(len(X_test), X_test.shape[1],1)

In [ ]:
target_test=test["Class"]
y_train=y_train
y_test=target_test


In [ ]:
y_train= np.array(y_train).reshape(-1,1)
y_test= np.array(y_test).reshape(-1,1)
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
encoder_df1 = pd.DataFrame(enc.fit_transform(y_train).toarray())
encoder_df2 = pd.DataFrame(enc.fit_transform(y_test).toarray())
y_train = np.array(encoder_df1)
y_test = np.array(encoder_df2)

In [ ]:
class_labels_ytr = np.argmax(y_train, axis=1)
class_labels_yte = np.argmax(y_test, axis=1)


In [ ]:
X_train.shape

(300000, 186, 1)

In [ ]:
n_obs, feature, depth = X_train.shape
batch_size = 500

In [ ]:
X_test.shape

(29872, 186, 1)

In [ ]:
y_train.shape

(300000, 15)

In [ ]:
y_test.shape

(29872, 15)

In [ ]:
X_train=X_train.reshape(300000, 186,1)
X_test=X_test.reshape(29872, 186,1)

In [ ]:
def exp_decay(epoch):
    initial_lrate = 0.001
    k = 0.75
    t = n_obs//(10000 * batch_size)  # every epoch we do n_obs/batch_size iteration
    lrate = initial_lrate * np.math.exp(-k*t)
    return lrate

In [ ]:
optimizer = Adadelta(lr=.001)

In [ ]:
lrate = LearningRateScheduler(exp_decay)

In [ ]:
my_callbacks = [
#    keras.callbacks.EarlyStopping(monitor='val_loss' , patience=8),
    keras.callbacks.ModelCheckpoint(filepath='cnn_model.h5' , monitor = 'val_loss' , save_best_only=True),
    lrate
]

In [ ]:
y_train=y_train[0:300000]
y_test=y_test[0:29872]

In [ ]:
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

(300000, 15)
(300000, 186, 1)
(29872, 15)
(29872, 186, 1)


In [ ]:
'''
# Applying the function to training set labels and testing set labels
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train )
y_test = to_categorical(y_test)
'''

'\n# Applying the function to training set labels and testing set labels\nfrom tensorflow.keras.utils import to_categorical\ny_train = to_categorical(y_train )\ny_test = to_categorical(y_test)\n'

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold ,  StratifiedKFold
import numpy as np

In [ ]:
acc_per_fold = []
loss_per_fold = []

inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)


In [ ]:
print(inputs.shape)

(329872, 186, 1)


In [ ]:
print(targets.shape)

(329872, 15)


In [ ]:

skf = StratifiedKFold(n_splits=5, shuffle=True)

fold_no = 1
for train , test in skf.split(inputs, np.argmax(targets,axis=1)):
    
  model = Sequential()

  model.add(Convolution1D(filters=32, kernel_size=5, strides=1 , input_shape=(186,1)))
  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
  model.add(MaxPooling1D(pool_size=5, strides=2))

  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
  model.add(MaxPooling1D(pool_size=5, strides=2))

  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
  model.add(MaxPooling1D(pool_size=5, strides=2))

  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
  model.add(MaxPooling1D(pool_size=5, strides=2))

  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
  model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
  model.add(MaxPooling1D(pool_size=5, strides=2))

  model.add(Flatten())

  model.add(Dense(512, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(256, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(64, activation='relu'))
  model.add(BatchNormalization())


  model.add(Dense(15, activation='softmax'))

  model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
 
  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=500,
              epochs=75,
              verbose=1)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/75
528/528 [==============================] - 20s 16ms/step - loss: 2.4122 - accuracy: 0.2783
Epoch 2/75
528/528 [==============================] - 8s 15ms/step - loss: 1.6341 - accuracy: 0.5272
Epoch 3/75
528/528 [==============================] - 8s 15ms/step - loss: 1.3259 - accuracy: 0.6458
Epoch 4/75
528/528 [==============================] - 8s 15ms/step - loss: 1.1411 - accuracy: 0.7031
Epoch 5/75
528/528 [==============================] - 8s 15ms/step - loss: 1.0116 - accuracy: 0.7424
Epoch 6/75
528/528 [==============================] - 8s 15ms/step - loss: 0.9108 - accuracy: 0.7735
Epoch 7/75
528/528 [==============================] - 8s 15ms/step - loss: 0.8330 - accuracy: 0.7958
Epoch 8/75
528/528 [==============================] - 8s 15ms/step - loss: 0.7703 - accuracy: 0.8123
Epoch 9/75
528/528 [==============================] - 8s 15ms/step - loss: 0.7185 - accuracy: 

In [ ]:
from statistics import mean
mean(acc_per_fold)

95.76835870742798

In [ ]:

from numpy import std
# Accuracy in SKFold
acc_per_fold


[95.63319683074951,
 96.21068835258484,
 95.48307061195374,
 95.63161134719849,
 95.88322639465332]

In [ ]:
print('min value element : ')
min(acc_per_fold)


min value element : 


95.48307061195374

In [ ]:
print("Max value element : ")
max(acc_per_fold)

Max value element : 


96.21068835258484

In [ ]:
import statistics
st_dev = statistics.pstdev(acc_per_fold)
print("Standard deviation of the given list: " + str(st_dev))

Standard deviation of the given list: 0.2558155473810743


In [ ]:
# loss in SKFold
loss_per_fold

[0.17708514630794525,
 0.15541546046733856,
 0.18254275619983673,
 0.18111537396907806,
 0.17264527082443237]

In [ ]:
print('mean in loss: ')
mean(loss_per_fold)

mean in loss: 


0.1737608015537262

In [ ]:
print('min value element : ')
min(loss_per_fold)




min value element : 


0.15541546046733856

In [ ]:
print("Max value element : ")
max(loss_per_fold)

Max value element : 


0.18254275619983673

In [ ]:
import statistics
st_dev = statistics.pstdev(loss_per_fold)
print("Standard deviation of the given list: " + str(st_dev))

Standard deviation of the given list: 0.009798568050973589
